In [2]:
!pip install pydmd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 6.3 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydmd import HODMD
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# -------------------------
# User settings / params
# -------------------------
CSV_PATH = 'DEMAND_08_NSW_2004.csv'
TARGET_DAY = pd.to_datetime("2004-08-22")
F = 48   # forecast horizon (48 half-hour steps)
PLOT_RESULTS = True

# -------------------------
# Utilities
# -------------------------
def get_day_values(df, day):
    """Return 48 half-hourly TOTALDEMAND values for given date or zeros if missing."""
    try:
        vals = df.loc[day.strftime('%Y-%m-%d'), 'TOTALDEMAND'].values.astype(float)
        if vals.shape[0] != 48:
            vals = np.resize(vals, 48)
        return vals
    except KeyError:
        print(f"Missing data for {day.strftime('%Y-%m-%d')}, returning zeros")
        return np.zeros(48, dtype=float)

def normalize(x):
    amin = np.min(x)
    amax = np.max(x)
    if amax == amin:
        return np.zeros_like(x), amin, amax
    return (x - amin) / (amax - amin), amin, amax

def denormalize(x_norm, amin, amax):
    if amax == amin:
        return np.full_like(x_norm, amin)
    return x_norm * (amax - amin) + amin

def calculate_mape(actual, predicted):
    actual = np.asarray(actual, dtype=float)
    predicted = np.asarray(predicted, dtype=float)
    mask = actual != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100

def diagonal_averaging(X):
    rows, cols = X.shape
    rev = X[::-1, :]
    diags = [rev.diagonal(i) for i in range(-rows + 1, cols)]
    diags_mean = np.array([np.mean(d) for d in diags])
    return diags_mean

# -------------------------
# Read CSV and prepare data
# -------------------------
df = pd.read_csv(CSV_PATH)
df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])
df = df.sort_values('SETTLEMENTDATE').set_index('SETTLEMENTDATE')

# Define the 4 input days
day_minus_1 = TARGET_DAY - pd.Timedelta(days=1)
day_minus_2 = TARGET_DAY - pd.Timedelta(days=2)
week_same_day = TARGET_DAY - pd.Timedelta(days=7)
week_day_before = TARGET_DAY - pd.Timedelta(days=8)
training_days = [day_minus_2, day_minus_1, week_day_before, week_same_day]

# Extract snapshots per day
snapshots_days = [get_day_values(df, d) for d in training_days]
actual_target = get_day_values(df, TARGET_DAY)

# -------------------------
# Make 2-day subsets (consecutive days)
# -------------------------
subsets = [
    np.concatenate(snapshots_days[0:2]),  # day_minus_2 + day_minus_1
    np.concatenate(snapshots_days[2:4])   # week_day_before + week_same_day
]

# -------------------------
# Evaluate function for given d and snapshot subset
# -------------------------
def evaluate_d(d, snapshots_subset):
    d = int(d)
    snaps_norm, amin, amax = normalize(snapshots_subset)

    model = HODMD(svd_rank=0, exact=True, opt=True, d=d)
    model.fit(snaps_norm[None, :])

    # Extend time for future prediction
    n_train = len(snaps_norm)
    model.dmd_time['tend'] = n_train + F - 1
    recon_norm_full = model.reconstructed_data.real[0]
    if recon_norm_full.shape[0] < n_train + F:
        recon_norm_full = np.pad(recon_norm_full, (0, n_train + F - recon_norm_full.shape[0]), 'edge')

    recon_denorm_full = denormalize(recon_norm_full, amin, amax)

    # Diagonal averaging
    G = max(1, n_train - F + 1)
    X_P = np.zeros((G, F))
    for i in range(G):
        X_P[i, :] = recon_denorm_full[i:i+F]

    diags_mean = diagonal_averaging(X_P)
    final_forecast = np.resize(diags_mean[:F], F)
    return calculate_mape(actual_target, final_forecast), final_forecast, model

# -------------------------
# Subset-based d search
# -------------------------
best_d_per_subset = []
forecasts_per_subset = []
models_per_subset = []

d_search_range = range(2, 50, 2)  # can adjust

# print("🔍 2-Day Subset-based d search...")
for idx, snaps in enumerate(subsets):
    best_d_subset = min(d_search_range, key=lambda d: evaluate_d(d, snaps)[0])
    mape_best, forecast_best, model_best = evaluate_d(best_d_subset, snaps)
    best_d_per_subset.append((best_d_subset, mape_best))
    forecasts_per_subset.append(forecast_best)
    models_per_subset.append(model_best)
    print(f"Subset {idx+1}: Best d = {best_d_subset}, MAPE = {mape_best:.3f}")

# Compare all subsets and select overall best d
best_d, best_mape = min(best_d_per_subset, key=lambda x: x[1])
best_idx = [i for i, (dval, mape) in enumerate(best_d_per_subset) if dval==best_d][0]
final_forecast = forecasts_per_subset[best_idx]
best_model = models_per_subset[best_idx]

print("\n✅ Final chosen d across all subsets:", best_d)
print(f"MAE = {np.mean(np.abs(actual_target - final_forecast)):.2f}, RMSE = {np.sqrt(mean_squared_error(actual_target, final_forecast)):.2f}, MAPE = {best_mape:.2f}")


Subset 1: Best d = 12, MAPE = 15.650
Subset 2: Best d = 30, MAPE = 6.628

✅ Final chosen d across all subsets: 30
MAE = 526.59, RMSE = 820.16, MAPE = 6.63
